In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk.stem
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from datetime import datetime
import string

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/zy2292/Wine-Master/master/Data/wine_data_cleaned.csv')
wine.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,year,unique_name
0,1,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87.0,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013.0,Nicosia 2013 Vulkà Bianco (Etna) White Blend
1,2,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87.0,15.0,Douro,NaN,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011.0,Quinta dos Avidagos 2011 Avidagos Red (Douro) ...
2,3,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013.0,Rainstorm 2013 Pinot Gris (Willamette Valley) ...
3,4,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013.0,St. Julian 2013 Reserve Late Harvest Riesling ...
4,5,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012.0,Sweet Cheeks 2012 Vintner's Reserve Wild Child...


In [3]:
print(str(datetime.now()))
stemmer = nltk.stem.SnowballStemmer('english')

def stem_tokens(tokens, stemmer):
    stemmed = map(lambda x: stemmer.stem(x), tokens)
    return stemmed

def tokenize(text):
    lowers = text.lower()
    lowers=lowers.replace("-"," ")
    no_punctuation = lowers.translate(str.maketrans('','',string.punctuation))
    tokens = nltk.word_tokenize(no_punctuation)
    stems = stem_tokens(tokens, stemmer)
    return stems

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english',
                          ngram_range=(1,1), max_df=0.95, min_df=0.025,
                          use_idf=True, binary=False, norm=None)
tfidf_matrix = tfidf.fit_transform(wine['description'])
print(str(datetime.now()))
print(tfidf_matrix.shape)

2018-04-18 15:15:59.714452
2018-04-18 15:17:24.169808
(116759, 189)


In [4]:
print(tfidf.get_feature_names())

['accent', 'acid', 'add', 'age', 'alcohol', 'almond', 'alongsid', 'appl', 'apricot', 'aroma', 'aromat', 'attract', 'bake', 'balanc', 'barrel', 'beauti', 'berri', 'best', 'big', 'bit', 'bitter', 'black', 'blackberri', 'blend', 'blueberri', 'bodi', 'bottl', 'bright', 'bring', 'cabernet', 'candi', 'cassi', 'charact', 'chardonnay', 'cherri', 'chocol', 'cinnamon', 'citrus', 'clean', 'close', 'clove', 'coffe', 'cola', 'color', 'come', 'complex', 'concentr', 'core', 'creami', 'crisp', 'crush', 'currant', 'dark', 'delic', 'delici', 'deliv', 'dens', 'domin', 'dri', 'drink', 'earth', 'earthi', 'easi', 'edg', 'eleg', 'end', 'enjoy', 'feel', 'fine', 'finish', 'firm', 'flavor', 'floral', 'flower', 'follow', 'fresh', 'fruit', 'fruiti', 'generous', 'good', 'grape', 'grapefruit', 'great', 'green', 'herb', 'herbal', 'high', 'hint', 'honey', 'impress', 'intens', 'juici', 'just', 'layer', 'lead', 'leather', 'lemon', 'licoric', 'light', 'like', 'lime', 'linger', 'littl', 'live', 'long', 'make', 'matur', '

In [5]:
import numpy as np
from itertools import chain
def get_recommendations(train, query):
    cos_query=cosine_similarity(train,query)
    cos_query=list(chain(*cos_query))
    sort_dist=np.argsort(cos_query)[::-1]
    return sort_dist[1:6].tolist()

In [7]:
strategy=[*map(lambda x: get_recommendations(tfidf_matrix,x),tfidf_matrix[77662])]

In [8]:
strategy

[[99224, 15491, 45437, 64740, 61945]]

In [9]:
wine.loc[77662,'description']

"There's plenty of ripe pie-filling fruit flavor in this dry wine. Cherries, blackberries, boysenberries and all sorts of other red fruits and berries mingle with melted chocolate, licorice and peppery spice. Turns a bit hot on the finish."

In [10]:
wine.loc[99224,'description']

"This 2013 keeps up the full body and ripe fruit of previous releases, with lush texture and saturated chocolate and blackberry flavors. It's a dry red, however, with no detectable sweetness but plenty of tannin and satisfaction."

In [11]:
wine.loc[15491,'description']

"Spice, licorice and herbal notes complement red-fruit aromas, while the palate offers plenty of structure and tannic grab. There's an avalanche of blackberry, cassis, fig, chocolate and herbal flavors to process, while the finish is rich and rewarding before breaking up in bits."

In [12]:
wine.loc[45437,'description']

"This superrich Zin has extracted, luxurious fruit. It's ripe in blackberry and cherry jam, milk chocolate, peppery spice and anise flavors, with an overripe note of Port. Drink now."

In [13]:
wine.loc[64740,'description']

"Full, forward and loaded with ripe strawberry and cherry flavors, this immediately likeable wine offers plenty of fruit and spice. It's almost chunky it's so big, with a lick of chocolate highlighting the finish."

In [14]:
wine.loc[61945,'description']

"Stainless steel fermentation has yielded a yeasty, beery wine. There's plenty of fruit, a bit too reminiscent of canned peaches, and a finish that seems constricted and short. Residual sugar is right around 2%."